In [15]:
import pandas as pd

from sudulunu.helpers import pp, dumper
import pathlib 
import os 
import json

rename = {
		"NSW": 'New South Wales',
		"QLD": 'Queensland',
		'TAS': 'Tasmania',
		'VIC':'Victoria',
		'WA':'Western Australia',
		'NT':'Northern Territory',
		'SA':'South Australia',
		"ACT": 'Australian Capital Territory'}

In [16]:
data = pd.read_csv('input/combined.csv')
# ['Date', 'Day', 'State', 'Weekday']

data['Year'] = pd.to_datetime(data['Date']).dt.strftime("%Y")

data['Count'] = 1

dicto = {}

for year in data['Year'].unique().tolist():
    inter = data.loc[data['Year'] == year].copy()
    bet = {}

    # uniques = inter.copy()
    # uniques.drop_duplicates(subset=['Date'], inplace=True)
    # pp(uniques)
    # grp = uniques.groupby(by=['Weekday'])['Count'].sum().reset_index()
    # bet['Australia'] = grp.to_json(orient='records')

    for state in inter['State'].unique().tolist():
        nother = inter.loc[inter['State'] == state].copy()
        nother.drop_duplicates(subset=['Date'], inplace=True)

        # Find holidays with "(observed)" suffix
        observed_holidays = nother[nother['Day'].str.contains(r'\(observed\)', case=False, na=False)]

        # For each observed holiday, find and remove the non-observed weekend version
        for _, obs_row in observed_holidays.iterrows():
            # Extract base holiday name (remove " (observed)")
            base_name = obs_row['Day'].replace(' (observed)', '').replace(' (Observed)', '')

            # Find the non-observed version that falls on weekend
            weekend_version = nother[
                (nother['Day'] == base_name) &
                (nother['Weekday'].isin(['Saturday', 'Sunday']))
            ]

            # Drop the weekend version
            if not weekend_version.empty:
                nother = nother.drop(weekend_version.index)

        grp = nother.groupby(by=['Weekday'])['Count'].sum().reset_index()
        bet[rename[state]] = grp.to_json(orient='records')
    
    dicto[year] = bet

print(dicto)
with open('calculator/static/state_counts.json', 'w') as f:
    json.dump(dicto, f, indent=4)

{'2025': {'Australian Capital Territory': '[{"Weekday":"Friday","Count":3},{"Weekday":"Monday","Count":6},{"Weekday":"Saturday","Count":1},{"Weekday":"Sunday","Count":1},{"Weekday":"Thursday","Count":1},{"Weekday":"Wednesday","Count":1}]', 'New South Wales': '[{"Weekday":"Friday","Count":3},{"Weekday":"Monday","Count":4},{"Weekday":"Saturday","Count":1},{"Weekday":"Sunday","Count":1},{"Weekday":"Thursday","Count":1},{"Weekday":"Wednesday","Count":1}]', 'Northern Territory': '[{"Weekday":"Friday","Count":3},{"Weekday":"Monday","Count":5},{"Weekday":"Saturday","Count":1},{"Weekday":"Sunday","Count":1},{"Weekday":"Thursday","Count":1},{"Weekday":"Wednesday","Count":1}]', 'Queensland': '[{"Weekday":"Friday","Count":3},{"Weekday":"Monday","Count":4},{"Weekday":"Saturday","Count":1},{"Weekday":"Sunday","Count":1},{"Weekday":"Thursday","Count":1},{"Weekday":"Wednesday","Count":2}]', 'South Australia': '[{"Weekday":"Friday","Count":3},{"Weekday":"Monday","Count":5},{"Weekday":"Saturday","Count

In [17]:
### Monday is the most common day overall

# grp = data.groupby(by=['Weekday'])['Count'].sum().reset_index()
# 0     Friday     68
# 1     Monday    169
# 2   Saturday     54
# 3     Sunday     40
# 4   Thursday     16
# 5    Tuesday     36
# 6  Wednesday     20

# pp(grp)

In [18]:
# ### Monday is the most common day next year

# grp = data.groupby(by=['Year', 'Weekday'])['Count'].sum().reset_index()

# grp = grp.loc[grp['Year'] == '2026']

# pp(grp)

In [19]:
### 

grp = data.groupby(by=['Year', 'State', 'Weekday'])['Count'].sum().reset_index()

grp = grp.loc[(grp['Year'] == '2026') & (grp['State'] == 'VIC')]


pp(grp)

    Year State   Weekday  Count
76  2026   VIC    Friday      3
77  2026   VIC    Monday      5
78  2026   VIC  Saturday      3
79  2026   VIC    Sunday      1
80  2026   VIC  Thursday      1
81  2026   VIC   Tuesday      1
['Year', 'State', 'Weekday', 'Count']


In [20]:
vic = data.loc[data['State'] == 'VIC'].copy()
vic = vic.loc[vic['Year'] == '2026']

pp(vic)

           Date                    Day State   Weekday  Year  Count
310  2026-04-03            Good Friday   VIC    Friday  2026      1
311  2026-04-06          Easter Monday   VIC    Monday  2026      1
312  2026-01-01         New Year's Day   VIC  Thursday  2026      1
313  2026-01-26          Australia Day   VIC    Monday  2026      1
314  2026-03-09             Labour Day   VIC    Monday  2026      1
315  2026-04-04        Easter Saturday   VIC  Saturday  2026      1
316  2026-04-05          Easter Sunday   VIC    Sunday  2026      1
317  2026-04-25              ANZAC Day   VIC  Saturday  2026      1
318  2026-06-08        King's Birthday   VIC    Monday  2026      1
319  2026-09-25        Grand Final Day   VIC    Friday  2026      1
320  2026-11-03      Melbourne Cup Day   VIC   Tuesday  2026      1
321  2026-12-25          Christmas Day   VIC    Friday  2026      1
322  2026-12-26             Boxing Day   VIC  Saturday  2026      1
323  2026-12-28  Boxing Day (observed)   VIC    